# Workshop 06

<b>Obiettivo</b>: Tradurre il codice del modello geometrico di <i>S. Stefano Rotondo al Celio</u>.

In [ ]:
# PLASM:

%=============================================================%
%                                                             %
%       Modello geometrico di S. Stefano Rotondo al Celio     %
%               Modello della Gerusalemme Celeste             %
%                                                             %
%                                                             %
%                       A.Paoluzzi, 1998                      %
%                                                             %
%=============================================================%

%*************************************************************%
%---Misure di riferimento--(cubito = 0.462 m)-----------------%
%*************************************************************%

%-- 1 cubito = 444 mm ----------------------------------------%
DEF r0 = 24; %---cubiti; raggio interno del muro interno------%
DEF r2 = 48; %---cubiti; raggio esterno del muro medio--------%
DEF r3 = 72; %---cubiti; raggio esterno del muro esterno------%
DEF r4 = 88; %---cubiti; raggio esterno scale esterne---------%

DEF wsteps = 10;  %---cubiti; profondita' scale esterne-------%
DEF nsteps = 10;  %---numero alzate/pedate scale esterne------%
DEF wstep = wsteps/nsteps; %---cubiti; larghezza pedata-------%
DEF hstep = 1/3; %---cubiti; altezza alzata-------------------%
DEF hw4 = 29; %--cubiti; altezza all'imposta del muro esterno-%
DEF hbasament = hstep*nsteps;

In [11]:
# PYPLASM:

# Importo le librerie:
import sys
sys.path.insert(0, 'C:\Users\Camilla Bianca\Documents\GGPL\lar-cc\larlib')
from larlib import *
import math

In [2]:
r0 = 24
r2 = 48
r3 = 72
r4 = 88

wsteps = 10
nsteps = 10
wstep = wsteps/nsteps
hstep = 1/3
hw4 = 29
hbasament = hstep*nsteps

In [ ]:
# PLASM:

%-------------------------------------------------------------%
%---Mappa cilindrica------------------------------------------%
%-------------------------------------------------------------%

DEF cylMap = MAP:[S2 * sin ~ S1, S2 * cos ~ S1, S3];

In [73]:
# PYPLASM:
import math 
fx = lambda x : x[1] * sin(x[0])
fy = lambda x : x[1] * cos(x[0])
fz = lambda x : x[2]

cylMap = MAP([fx,fy,fz])

In [51]:
# PLASM:

%-------------------------------------------------------------%
%---Scala esterna---------------------------------------------%
%-------------------------------------------------------------%

DEF vdom (h::IsRealPos) = ((EMBED:1 ~ Intervals:(PI*3/24)):3) * QUOTE:<h>;
DEF hdom (w::IsRealPos) = (S:2:-1 ~ EMBED:1):(Intervals:(PI*3/24):3 * QUOTE:<w>);

DEF steps (w,h::IsRealPos) (n::IsIntPos) = (STRUCT ~ CAT ~ #:n):
  < vdom:h, T:3:h, hdom:w, T:2:(-:w) >;

DEF hbasament = hstep*nsteps;
DEF stair = (MKPOL ~ UKPOL):(steps:<wstep,hstep>:nsteps  
              RIGHT (S:2:-1 ~ @2 ~ CUBOID):<PI/24,wsteps,hbasament>);
DEF ramp  = cylMap:(T:2:r4:stair);
DEF stairs = (STRUCT ~ ##:12):<ramp, R:<1,2>:(PI/6)>;

<function pyplasm.fenvs.<lambda>>

In [85]:
# PYPLASM:

def vdom (h):
    return EMBED(1)(INTERVALS(PI*3/24)(3))*QUOTE(h)

def hdom(w):
    S(2)(-1)(EMBED(1)(INTERVALS(PI*3/24)(3))*QUOTE(w))

def steps (w,h):
    def steps1(n):
        return COMP([STRUCT,CAT,N(n)])([vdom(h), T(3)(h), hdom(w), T(2)(-w)])
    return steps1

hbasement = hstep*nsteps

stair = RIGHT([COMP([MKPOL, UKPOL])(steps(wstep,hstep)(nsteps)),
               COMP([S(2)(-1),SKEL_2,CUBOID])([PI/24,wsteps,hbasement])
              ])

# CONTINUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


TypeError: 'int' object is not iterable

In [ ]:
# PLASM:

%-------------------------------------------------------------%
%---Basamento-------------------------------------------------%
%-------------------------------------------------------------%

DEF basisSector = (cylMap ~ EMBED:1):
    (Intervals:(2*PI/12):4 * Intervals:(r4 - (wstep*(nsteps - 1))):1);
DEF basis = (STRUCT ~ ##:12):<basisSector, R:<1,2>:(2*PI/12)>;
DEF basement = (R:<1,2>:(PI/-48) ~ STRUCT):< stairs, T:3:(hstep*nsteps), basis >;

%
basement
%

In [83]:
# PYPLASM:

basisSector = COMP([cylMap,EMBED(1)])(PROD([INTERVALS(2*PI/12)(4),INTERVALS(r4-(wstep*(nsteps-1)))(1)]))

basis = COMP([STRUCT,NN(12)])([basisSector,R([1,2])(2*PI/12)])

basement = R([1,2])(PI/-48)(STRUCT([stairs,T(3)(hstep*nsteps),basis]))

NameError: name 'stair' is not defined